In [ ]:
# Python multivariate analysis (Support Vector Classification) script
# written by Simon O'Callaghan NICTA
# 14 May 2015

import matplotlib
matplotlib.use('Agg')
rootPath='/notebooks/'
import sys
sys.path.append(rootPath+'bdkd-external-devel/rocks/seafloorLitho/')
sys.path.append(rootPath+'revrand-master/')
sys.path.append(rootPath+'bdkd-external-devel')
sys.path.append("/usr/local/lib/python3.4/site-packages/")

import os
import pandas as pd
import numpy as np
import utils
from sklearn.svm import SVC
from sklearn import cross_validation
from matplotlib import pyplot as pl
%matplotlib inline  
from multiprocessing import Pool
import multiprocessing
import scipy.linalg as linalg

def loadData(path):
    data = pd.read_csv(path)
    for i in range(len(data.columns)):
      data  = data[np.isfinite(data[data.columns[i]])]  # Remove any nans

    # Set up the features (X) and the targets or labels (y)
    X = data[data.columns[3:]].values
    y = data.lithology.values
    lonlat = data[data.columns[:2]].values
    return X-np.min(X,axis=0),y, lonlat


def preprocessData(X):
    X[:, 4] = np.log(X[:, 4]+1e-20)
    X[:, 6] = np.log(X[:, 6]+1e-20)
    return X

def normaliseData(X):
    means = np.mean(X, axis=0)
    std_devs = np.std(X, axis=0)
    X = (X - means)/std_devs
    return X

def printMostUsefulFeatures(X, y, nRandomFeatures, featOnListArray, resultsList,maxSampPerClass, lithoLabel):
    results = np.asarray(resultsList).T
    nFeatures = X.shape[1]
    nTrueFeatures = nFeatures - nRandomFeatures
    classifierScoreList=[]

    XCull, yCull = utils.cullData(X,y,maxSampPerClass)
    featureAccuracy = np.zeros((nTrueFeatures,results.shape[1]))
    for nFeaturesIter in (np.arange(nTrueFeatures)+1):
        classifierScore = np.mean(results[np.sum(featOnListArray, axis=1)==nFeaturesIter], axis=1)
        classifierScoreList.append(np.max(classifierScore).astype(float))
        activeFeatures = featOnListArray[np.sum(featOnListArray,axis=1)
                                             ==nFeaturesIter,:]
        bestFeatures = np.arange(nFeatures)[activeFeatures[np.argmax(classifierScore)]==1]+1
        bestFeatureID = np.argmax(classifierScore)
        featureAccuracy[(nFeaturesIter-1),:] = results[np.sum(featOnListArray, axis=1)==nFeaturesIter][bestFeatureID,:]
        print('For a classifier using %d features,' %(nFeaturesIter))
        print('the most informative features are:')
        print(bestFeatures.astype(list), 'with a score of %f' %(np.max(classifierScore)))
        scoreList = []
        for iter in range(results.shape[1]):
            Xrand = np.random.randn(XCull.shape[0], nFeaturesIter)
            X_train, X_test, y_train, y_test = cross_validation.train_test_split(
              Xrand, yCull, test_size=0.2)
            svc = SVC(kernel='rbf', probability=True)
            svc.fit(X_train,y_train)
            scoreList.append(svc.score(X_test, y_test))

        print('%d random features have a score of: %f' %(nFeaturesIter, np.mean(scoreList)))
        print()

    pl.figure()
    pl.errorbar((np.arange(nTrueFeatures)+1).astype(int), np.mean(featureAccuracy,axis=1), np.std(featureAccuracy,axis=1))
    pl.title('Area Under ROC Curve vs Number of Features')
    pl.xlabel('Number of features')
    pl.ylabel('AUC')
    xint = range(0,nTrueFeatures+2)
    pl.xticks(xint)
    ax = pl.gca()
    ax.set_xlim((0,nTrueFeatures+1))
    pl.savefig('auc'+str(lithoLabel)+'.pdf')



def genFeatureFreq(nFeatures, nRandomFeatures, results,featOnArray):
    nTrueFeatures = nFeatures - nRandomFeatures
    featureFreq = np.zeros([nTrueFeatures, nFeatures])

    for scoreList in results:
        maxFeatSortedID = np.argsort(scoreList)[::-1]
        maxFeatSorted = featOnArray[maxFeatSortedID, :]

        for nFeaturesIter in (np.arange(nTrueFeatures)+1):
              indexBoole = np.sum(maxFeatSorted,axis=1)==nFeaturesIter
              featureFreq[nFeaturesIter-1,:] = featureFreq[nFeaturesIter-1,:] + \
                                               maxFeatSorted[indexBoole,:][0,:]

    return featureFreq

def genAUCMatrix(nFeatures, nRandomFeatures, results,featOnArray):

    results = np.mean(results,axis=0)

    nTrueFeatures = nFeatures - nRandomFeatures
    featureFreq = np.zeros([nTrueFeatures, nFeatures])

    for i, activeFeats in enumerate(featOnArray[:-1,:]):
        for j in np.arange(np.sum(activeFeats)):
            col_id = np.where(activeFeats==1.)[0]
            featureFreq[np.sum(activeFeats)-1,col_id[j]] = results[i]

    return featureFreq


def plotMatrix(featureFreq,lithoLabel):
    nTrueFeatures = featureFreq.shape[0]
    nFeatures = featureFreq.shape[1]
    pl.figure()
    pl.imshow(featureFreq, interpolation='nearest', extent=(0.5, nFeatures + 0.5, nTrueFeatures + 0.5, 0.5))
    pl.xlabel('Feature ID')
    pl.ylabel('Number of Features')
    pl.colorbar()
    ax = pl.gca()
    ax.tick_params(axis='x', top='off')
    ax.tick_params(axis='y', right='off')
    ax.get_yaxis().set_tick_params(direction='out')
    ax.get_xaxis().set_tick_params(direction='out')
    pl.savefig('balancedmultvarAnalysisLabel'+str(lithoLabel)+'Seafloor.pdf')


def removeCorrelatedFeatures(X):
    keepdims = [0,6,4,5,7, -1]
    X = X[:,keepdims]

    return X

def removeHemisphere(X, y_map, lonlat):
    keeppoints_id= np.where(lonlat[:,1]<0)[0]

    return X[keeppoints_id,:], y_map[keeppoints_id]


def mainscript(lithoLabelList):

    for lithoLabel in lithoLabelList:

        # Set up Variables
        path = rootPath+'bdkd-external-devel/rocks/seafloorLitho/seafloor_lith_data_all_features.csv'
        nIterations = 5
        nRandomFeatures = 1

        removeLabels = [9,10,11,12]

        # Load Data
        X,y, lonlat = loadData(path)

        # Remove a hemisphere from the data to analyse effects
        X, y = removeHemisphere(X, y, lonlat)

        # Preprocess the data
        X = preprocessData(X)

        # Adding random features to X
        X = np.append(X, np.random.random([X.shape[0], nRandomFeatures]), axis=1)

        # Normalise Data
        X = normaliseData(X)

        # Remove correlated features
        X = removeCorrelatedFeatures(X)

        # Map labels to groups
        mapping = {'1': 1, '2': 1, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 5,
                   '9': 9, '10': 10, '11': 11, '12': 12, '13': 3}
        y_map = np.zeros(y.shape)
        for i in range(y.shape[0]):
            y_map[i] = mapping[str(int(y[i]))]

        # Remove labels as requested by geoscientist
        for remLab in removeLabels:
            X = X[y_map != remLab,:]
            y_map = y_map[y_map != remLab]

        X, y_map = utils.balanceData(X, y_map)

        y_map[y_map != lithoLabel] = 0
        y_map[y_map == lithoLabel] = 1
        nFeatures = X.shape[1]

        maxSampPerClass = np.min([2000, int(np.sum(y_map))])

        # Generature all permutations of the features
        featureArray = utils.genFeatureArray(nFeatures)


        # Set up a core pool and evaluate the performance of each permutation nIteration times
        pool = Pool(processes=(multiprocessing.cpu_count()-1))
        results = pool.map(utils.evalFeaturesGP, [(X,y_map,featureArray,maxSampPerClass) for i in range(nIterations)])

        # Print the best performing permutations
        printMostUsefulFeatures(X,y_map, nRandomFeatures,featureArray,results, maxSampPerClass, lithoLabel)

        # Generate a matrix which shows how frequently different features were used depending on
        # the number of features provided
        featureFreq = genFeatureFreq(nFeatures, nRandomFeatures, results, featureArray)

        np.savetxt('balancedgroup'+ str(lithoLabel)+'LithoOutput_Label.txt',
               featureFreq)
        plotMatrix(featureFreq/nIterations,lithoLabel)

    pl.show()

if __name__ == '__main__':
    lithoLabelList = [5]
    mainscript(lithoLabelList)


